In [1]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install selenium
# set options to be headless, ..
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# open it, go to a website, and get results
driver = webdriver.Chrome('chromedriver',options=options)

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [697 B]
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:7 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:8 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:12 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Ign:13 https://developer.download.nvidia.com/comp

In [2]:
# Import necessary packages
from selenium import webdriver
from string import punctuation
import numpy as np
import pandas as pd

In [15]:
# Scrape data from the website
hentaivn = pd.DataFrame(columns = ['Name','Categories', 'Views', 'Date',
                                     'Likes', 'Dislikes'])

for i in range(1021, 1101):
    link_page = 'https://hentaivn.net/danh-sach.html?page={}'.format(i)
    driver.get(link_page)
    list_info = driver.find_elements_by_xpath('//div[@class="box-description"]/p[1]/a')
    
    for j in range(16):
        try:
            # Extract name
            name = list_info[j].text
            # Extract href
            link_href = list_info[j].get_attribute('href')

            driver.get(link_href)
            # Extract categories
            categories = driver.find_elements_by_xpath('//a[@class="tag"]')
            for i in range(len(categories)):
                categories[i] = categories[i].text
        except:
            name = ''
            categories = []

        # Extract views
        def extract_views(s):
            start = 0
            end = 0
            for i in range(len(s)):
                if s[i].isnumeric():  
                    start = i
                    break
            for j in range(start, len(s)):
                if s[j] == ' ':
                    end = j
                    break
            return s[start:end]
        try:
            views = driver.find_elements_by_xpath("//span[contains(text(),'Lượt xem')]//parent::p")[0].text
            views = extract_views(views)
        except:
            views = 0

        # Extract date
        try:
            date = driver.find_elements_by_xpath("//*[contains(text(),'Update')]//parent::span/i")[0].text[8:]
        except:
            date = 0

        # Extract likes and unlikes
        try:
            likes = driver.find_elements_by_xpath('//div[@class="but_like"]')[0].text
            unlikes = driver.find_elements_by_xpath('//div[@class="but_unlike"]')[0].text
        except:
            likes = 0
            unlikes = 0

        hentaivn.loc[len(hentaivn)] = [name, categories, views, date, likes, unlikes]
        driver.get(link_page)
        list_info = driver.find_elements_by_xpath('//div[@class="box-description"]/p[1]/a')

In [16]:
hentaivn.to_csv('hentai(1021-1100).csv',index=False)